# Obtaining GPT-2 based Feature Representation

### 1. Imports and Global Settings

In [1]:
from datasets import load_dataset, disable_caching
from transformers import GPT2TokenizerFast, GPT2ForSequenceClassification, \
    DataCollatorWithPadding, set_seed
import torch
from torch.nn.functional import one_hot
from utils_ import tokenize, train_classifier
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
set_seed(42)
disable_caching()

### 2. Pre-Processing
- Get SNLI Dataset (Train fold) and shuffle it
- Remove instances without gold standard labels, i.e., label = -1
- One-hot encoding for labels
- Partition data 10%/90%; store the 10% as `warmup`
- Tokenise `warmup`

In [2]:
snli_train = load_dataset('snli', split = 'train').shuffle(seed = 42)
snli_train = snli_train.filter(lambda x: x['label'] != -1).map( \
    lambda x: {'label': one_hot(torch.tensor(x['label']), 3).type(torch.float32).numpy()}, \
    batched = True)
warmup = snli_train.select(range(0, int(len(snli_train)/10)))

Reusing dataset snli (/home/shana92/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b)


  0%|          | 0/551 [00:00<?, ?ba/s]

  0%|          | 0/550 [00:00<?, ?ba/s]

In [3]:
# set up tokeniser
# padding to left because GPT2 uses last token for prediction
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2", padding_side = 'left', \
                                              padding = True, truncation = True)
tokenizer.pad_token = tokenizer.eos_token # pad with 'eos' token

In [4]:
# tokenize data
warmup = warmup.map(lambda x: tokenize(tokenizer, x['premise'] + '|' + x['hypothesis']))
len_bef_exclusion = len(warmup)

# exclude instances with > 128 tokens
warmup = warmup.filter(lambda x: x['exclude'] == False)
len_aft_exclusion = len(warmup)

# print message if instances were in fact excluded
if len_bef_exclusion - len_aft_exclusion > 0:
    
    print(f'{len_bef_exclusion - len_aft_exclusion} ' + \
          f'({(len_bef_exclusion/len_aft_exclusion - 1)*100:>2f}%) sequences excluded')

  0%|          | 0/54936 [00:00<?, ?ex/s]

  0%|          | 0/55 [00:00<?, ?ba/s]

In [5]:
# keep only needed columns, set data format to PyTorch
warmup.set_format(type = 'torch', columns = ['label', 'input_ids', 'attention_mask'])

### 3. Obtain Feature Representation:

In [6]:
# set up data collator - https://huggingface.co/docs/transformers/main_classes/data_collator
# this is a (callable) helper object that sends batches of data to the model
data_collator = DataCollatorWithPadding(tokenizer, padding = 'max_length', \
                                         return_tensors = 'pt', max_length = 128)

In [7]:
# set up GPT2
model = GPT2ForSequenceClassification.from_pretrained("gpt2", 
                                  num_labels=3,
                                  problem_type="multi_label_classification")
model.config.pad_token_id = model.config.eos_token_id # specify pad_token used by tokenizer

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
# set up a dataloader (batch generator)
dataloader = torch.utils.data.DataLoader(warmup, batch_size=92, \
     shuffle=True, collate_fn=data_collator) # batch size  set to match Le et al. (2020) - https://arxiv.org/abs/2002.04108

In [9]:
# set up optimizer (loss function in-built)
# `lr` set to match Le et al. (2020) - https://arxiv.org/abs/2002.04108
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

In [10]:
# Train and save model
torch.save(train_classifier(model, dataloader, optimizer, device), 'feature_rep.pth')

  0%|          | 0/598 [00:00<?, ?it/s]

loss: 1.769108  [    0/54936]
loss: 0.641315  [ 5428/54936]
loss: 0.645410  [10856/54936]
loss: 0.653480  [16284/54936]
loss: 0.628841  [21712/54936]
loss: 0.641234  [27140/54936]
loss: 0.624826  [32568/54936]
loss: 0.624907  [37996/54936]
loss: 0.602843  [43424/54936]
loss: 0.605084  [48852/54936]
loss: 0.612408  [54280/54936]
Epoch average loss: 0.642099142074585


  0%|          | 0/598 [00:00<?, ?it/s]

loss: 0.613521  [    0/54936]
loss: 0.577154  [ 5428/54936]
loss: 0.612191  [10856/54936]
loss: 0.590110  [16284/54936]
loss: 0.585540  [21712/54936]
loss: 0.559032  [27140/54936]
loss: 0.604755  [32568/54936]
loss: 0.486844  [37996/54936]
loss: 0.488665  [43424/54936]
loss: 0.523453  [48852/54936]
loss: 0.513304  [54280/54936]
Epoch average loss: 0.5747400522232056


  0%|          | 0/598 [00:00<?, ?it/s]

loss: 0.514699  [    0/54936]
loss: 0.515072  [ 5428/54936]
loss: 0.506219  [10856/54936]
loss: 0.541514  [16284/54936]
loss: 0.537029  [21712/54936]
loss: 0.530551  [27140/54936]
loss: 0.415988  [32568/54936]
loss: 0.455232  [37996/54936]
loss: 0.501762  [43424/54936]
loss: 0.442487  [48852/54936]
loss: 0.443420  [54280/54936]
Epoch average loss: 0.4970329701900482
Done!


The feature representation $\Phi$ to be extracted (see report) is the saved model with the final linear layer removed